### Parse data from pdb50

Output is a dict with cluster number as key and values corresponding to the entries in this clusters.

In [1]:
blast50 = {}
with open('in/bc-50.out') as f:
    c = 0
    for line in f:
        c += 1
        line = line.rstrip()
        clean_data = []
        data = line.split(' ')
        for entry in data:
            pdb, chain = entry.split('_')
            pdb = pdb.lower()
            ident = '%s_%s' % (pdb, chain)
            clean_data.append(ident)
        blast50[c] = clean_data

### Match pdb50 identifiers with sequences based on pdb_seqres

Output is a dict with entry identificators as keys and sequences as values.

In [2]:
blast50_flatened = set()
for entries in blast50.values():
    for entry in entries:
        blast50_flatened.add(entry)

In [3]:
id2seq = {}
aa1 = "ACDEFGHIKLMNPQRSTVWY"
from Bio import SeqIO
for record in SeqIO.parse("/home/db/pdb/pdb_seqres_clean.txt", "fasta"):
    nonstd_present = False
    for aa in str(record.seq):
        if aa not in aa1:
            nonstd_present = True
            break
    if not nonstd_present and record.id in blast50_flatened:
        id2seq[record.id] = str(record.seq)
            

In [4]:
len(blast50_flatened), len(id2seq)

(395018, 293987)

# Get dataset

### Load Socket data from PDB Biounit run

cc_biounit_74_kih.p file was generated with the socket.py script

In [5]:
import pickle
ccdata = pickle.load(open('in/cc_biounit_74_kih.p', 'rb'))

In [6]:
len(ccdata)

11117

Data format example

In [7]:
ccdata['2caz']

([{'heptads': ['--------------------------', '-------------------------'],
   'indices': [('0', '328', '345', 'A'), ('1', '359', '380', 'A')],
   'oligomerization': '2',
   'orientation': 'antiparallel',
   'sequences': ['LYNLVAQDYALTDTIECL', 'FVKQGRELARQQFLVRWHIQRI']}],
 [['0', '1', 'antiparallel']])

### Get positive and negative sets - check if <font color="red">any</font> entry from each cluster contains coiled coil domain

This part iterates through entries in each pdb50 cluster to find a sequence containing a coiled coil domain based on the socket assignment. If CC domain is not found it returns the first entry in the cluster. Returned assignment is 0/1 (1 being CC present) and it's length is equal to the length of the sequence.

In [8]:
import re
import os
import itertools
to_del = set()
def find_index(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]
def change_X_seq(indexes, values, seq):
    seq = list(seq)
    for index, value in zip(indexes, values):
        seq[index] = value
    return ''.join(seq)
cc_allentries = {} ## All CC entries
nocc_allentries = {} ## All non-CC entries
c = 0
for entries in blast50.values():
    try:
        proper_entries = []
        cc_present = False
        for entry in entries:
            if entry in id2seq.keys():
                seq = id2seq[entry]
                if len(seq) > 30 and len(seq) < 1000: ## Only sequences of reasonable length
                    proper_entries.append(entry)
                    assignment = len(seq)*'0' ## CC assignment
                    pdb, chain = entry.split('_')
                    if pdb in ccdata.keys(): 
                        data = ccdata[pdb][0]
                        for cc in data:
                            for indice, cc_seq, heptad in zip(cc['indices'], cc['sequences'], cc['heptads']):
                                if indice[3]==chain:
                                    matched = False
                                    if 'X' in cc_seq:
                                        indexes = find_index(cc_seq, 'X')
                                        for perm in itertools.product('MCKH', repeat=len(indexes)):
                                            cc_seq2 = change_X_seq(indexes, perm, cc_seq)
                                            starts = [match.start() for match in re.finditer(re.escape(cc_seq2), id2seq[entry])]
                                            if starts:
                                                matched = True
                                                if len(cc_seq) >= 7: ## Only CC domains of length 7 or more
                                                    cc_present = True
                                                    for start in starts:
                                                        temp_list = list(assignment)
                                                        for i in range(0, len(cc_seq)):
                                                            temp_list[start+i] = '1'
                                                        assignment = ''.join(temp_list)
                                    else:
                                        starts = [match.start() for match in re.finditer(re.escape(cc_seq), id2seq[entry])]
                                        if starts:
                                            matched = True
                                            if len(cc_seq) >= 7: ## Only CC domains of length 7 or more
                                                cc_present = True
                                                for start in starts:
                                                    temp_list = list(assignment)
                                                    for i in range(0, len(cc_seq)):
                                                        temp_list[start+i] = '1'
                                                    assignment = ''.join(temp_list)
                                    if not matched:
                                        print('%s' % (entry), cc_seq, id2seq[entry])
                                        to_del.add(entry)
                        if cc_present:
                            raise IOError
        if not cc_present:
            if len(proper_entries) > 0:
                nocc_allentries[proper_entries[0]] = [id2seq[proper_entries[0]], len(id2seq[proper_entries[0]])*'0']
    except IOError:
        if matched:
            cc_allentries[entry]= [seq, assignment]

5hj3_F NRKAIDFL ELVMTQSPDSLAVSLGERATINCKSSQSVLYSSNNKSYLAWYQQKPGQPPKLLIYWASTRESGVPDRFSGSGSGTDFTLTISSLQAEDVAVYYCQQYYSAPLTFGGGTKVEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLRSPVTKSFNR
4fqk_F ILELDEKVDDLRADTISSQIELAVLL QSVLTQPPSASGTPGQRVTISCSGSSSNIGTNYVYWYQQFPGTAPKLLIYRSYQRPSGVPDRFSGSKSGSSASLAISGLQSEDEADYYCATWDDSLNGWVFGGGTKLTVLRQPKAAPSVTLFPPSSEELQANKATLVCLISDFYPGAVTVAWKADSSPVKAGVETTTPSKQSNNKYAASSYLSLTPEQWKSHRSYSCQVTHEGSTVEKTVAPTECS
4c58_B LKIFYQTCRAVQ QVQLQESGGGSVQAGGSLRLSCGASEYTSRMGWFRQAPGAEREGVACIHRQSNLSYYSDSVRGRFTISQDNAKTTAFLLMSSLKPEDTAIYYCATTTDCAAFVERATAITAGQGTQVTVSSAAAYPYDVPDYGSHHHHHH
4c58_B IAEVVHQLQEIA QVQLQESGGGSVQAGGSLRLSCGASEYTSRMGWFRQAPGAEREGVACIHRQSNLSYYSDSVRGRFTISQDNAKTTAFLLMSSLKPEDTAIYYCATTTDCAAFVERATAITAGQGTQVTVSSAAAYPYDVPDYGSHHHHHH
2x3x_D LERAWGAMMTEADKVS AKGVRVRALYDYDGQEQDELSFKAGDELTKLGEEDEQGWCRGRLDSGQLGLYPANYVEAI
2x3x_D VKRIDDGHRLCNDLMSCVQERAKIEKAYAQQLTDWAKRW AKGVRVRALYDYDGQEQDELSFKAGDELTKLGEEDEQGWCRGRLDSGQLGLYPANYVEAI


Data format example

In [9]:
to_del

{'2x3x_D', '2xz0_D', '4c58_B', '4fqk_F', '5hj3_F'}

In [10]:
len(cc_allentries)

3559

# Check for entries with PSSM and corresponding PSIBLAST out files

Double check if psiblast PSSM files are available for certain entry and that the sequence matches the sequence in the PSSM file

In [11]:
def get_pssm_sequence(fn):
    c = 0
    seq_list = []
    try:
        with open(fn) as f:
            for line in f:
                if c > 2:
                    try:
                        aa = line.split()
                        seq_list.append(aa[1])
                    except IndexError:
                        break
                c += 1
        f.close()
    except FileNotFoundError:
        pass
    return ''.join(seq_list)

In [13]:
cc_allentries_clean = {}
nocc_allentries_clean = {}

db_pssm3 = pickle.load(open('/home/db/psiblast/PSSM_3_IT.p', 'rb'))
for entry, values in cc_allentries.items():
    fn1 = '/home/db/psiblast/PSSM_3_IT/%s.pssm' % entry
    if entry in db_pssm3 and entry not in to_del:
        if get_pssm_sequence(fn1) == values[0]:
            cc_allentries_clean[entry] = values
for entry, values in nocc_allentries.items():
    fn1 = '/home/db/psiblast/PSSM_3_IT/%s.pssm' % entry
    if entry in db_pssm3 and entry not in to_del:
        if get_pssm_sequence(fn1) == values[0]:
            nocc_allentries_clean[entry] = values

# Counts

In [14]:
len(cc_allentries), len(nocc_allentries), len(cc_allentries)+len(nocc_allentries)

(3559, 25591, 29150)

In [15]:
len(cc_allentries_clean), len(nocc_allentries_clean), len(cc_allentries_clean)+len(nocc_allentries_clean)

(3558, 25381, 28939)

# Final checkup

In [16]:
for entry, value in nocc_allentries_clean.items():
    assert '1' not in value[1]
for entry, value in cc_allentries_clean.items():
    assert '1'  in value[1]

In [17]:
set(cc_allentries_clean.keys()) & set(nocc_allentries_clean.keys())

set()

# Create pandas dataframe

In [18]:
import pandas as pd

In [19]:
df1 = pd.DataFrame.from_dict(cc_allentries_clean, orient='index')
df1.columns = ['sequence', 'socket_assignment']
df1['cc'] = 1
df2 = pd.DataFrame.from_dict(nocc_allentries_clean, orient='index')
df2.columns = ['sequence', 'socket_assignment']
df2['cc'] = 0

In [20]:
df = pd.concat([df1, df2])

In [21]:
df.head()

,sequence,socket_assignment,cc
4cb9_A,MAHHHHHHMDVGELLSYQPNRGTKRPRDDEEEEQKMRRKQTGTRER...,0000000000000000000000000000000000000000000000...,1
1yo7_A,MTKQEKTALNMARFIRSQTLTLLEKLNELDADEQADICESLHDHAD...,0000000111111111111111111100000001111111111111...,1
3p0b_A,MGSSHHHHHHSSGLVPRGSHMARFALVLHAHLPYVRAHGMWPFGEE...,0000000000000000000000000000000000000000000000...,1
3cdh_A,SNAMNDTPDDTFVSGYLLYLLAASSEEASAQFHDHIRAQGLRVPEW...,0000000000000000011111111111111100000000000000...,1
3k9i_A,GVLGLEAQAVPYYEKAIASGLQGKDLAECYLGLGSTFRTLGEYRKA...,0000000000000000000000000000000000000000001111...,1


# Dump data

In [22]:
df.to_csv('./out/csv/data_all_74.csv')
df.to_pickle(open('./out/pickle/data_all_74.p', 'wb'))

In [23]:
df.shape

(28939, 3)